In [43]:
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm import tqdm

In [5]:
subdf = pd.read_excel('suburbs.xlsx')
subdf = subdf[['Unnamed: 0','state','domain_search_string']]
subdf.rename(columns = {'Unnamed: 0':'suburb_id','state':'state_code'},inplace=True)

In [9]:
mapcols = {'Northern Territory':'NT','New South Wales':'NSW','Australian Capital Territory':'ACT',
 'Victoria':'VIC','Queensland':'QLD','South Australia':'SA','Western Australia':'WA','Tasmania':'TAS'}
subdf['state'] = subdf['state_code'].replace(mapcols)

In [13]:
subdf.columns

Index(['suburb_id', 'state_code', 'domain_search_string', 'state'], dtype='object')

In [11]:
latdf = pd.read_excel('Latitude_and_Longitude_Extract_20230802.xlsx')

In [12]:
latdf.rename(columns={'lattitude':'latitude'},inplace=True)

In [15]:
latdf.rename(columns={'Search_string':'domain_search_string'},inplace=True)
latdf.drop(columns=['lat_and_lon'],inplace=True)
df = pd.merge(subdf,latdf,how = 'left',left_on='domain_search_string',right_on='domain_search_string')

In [32]:
df

,suburb_id,state_code,domain_search_string,state,latitude,longitude
0,1,Northern Territory,darwin-city-nt-0800,NT,-12.463733,130.844445
1,2,Northern Territory,alawa-nt-0810,NT,-12.381136,130.875298
2,3,Northern Territory,brinkin-nt-0810,NT,-12.370405,130.869843
3,4,Northern Territory,casuarina-nt-0810,NT,-12.372551,130.883775
4,5,Northern Territory,coconut-grove-nt-0810,NT,-12.398889,130.852500
...,...,...,...,...,...,...
15874,15875,Tasmania,granville-harbour-tas-7469,TAS,-41.807842,145.035194
15875,15876,Tasmania,renison-bell-tas-7469,TAS,-41.795235,145.435804
15876,15877,Tasmania,trial-harbour-tas-7469,TAS,-41.928422,145.175424
15877,15878,Tasmania,zeehan-tas-7469,TAS,-41.882997,145.335915


In [39]:
geolocator = Nominatim(user_agent="my_geocoder")
for index, row in df[(df['latitude'].isna() | df['longitude'].isna())].iterrows():
    suburb_name = (' '.join(row['domain_search_string'].split('-')[:-2]))
    location = geolocator.geocode(f"{suburb_name}, {row['state']}, Australia")
    if location:
        df.at[index, 'latitude'] = location.latitude
        df.at[index, 'longitude'] = location.longitude

In [ ]:
geolocator = Nominatim(user_agent="my_geocoder")

def get_city(latitude, longitude):
    if pd.isna(latitude) or pd.isna(longitude):
        return None
    try:
        location = geolocator.reverse((latitude, longitude), exactly_one=True)
        if location:
            return location.raw['address']['city']
    except Exception as e:
        print(f"Error: {e}")
    return None

# Filter out rows with NaN values in latitude or longitude columns
df = df.dropna(subset=['latitude', 'longitude'])

# Apply the function to each row in the DataFrame, printing progress
tqdm.pandas()  # Enable progress_apply method
df['city'] = df.progress_apply(lambda row: get_city(row['latitude'], row['longitude']), axis=1)


In [55]:
geolocator = Nominatim(user_agent="my_geocoder")

def get_city(latitude, longitude):
    if pd.isna(latitude) or pd.isna(longitude):
        return None
    try:
        location = geolocator.reverse((latitude, longitude), exactly_one=True)
        if location:
            return location.raw['address']
    except Exception as e:
        # print(f"Error: {e}")
        None
    return None

# Filter out rows with NaN values in latitude or longitude columns
df = df.dropna(subset=['latitude', 'longitude'])

# Apply the function to each row in the DataFrame, printing progress
tqdm.pandas()  # Enable progress_apply method
df['adress'] = df.progress_apply(lambda row: get_city(row['latitude'], row['longitude']), axis=1)


100%|██████████| 15859/15859 [2:13:54<00:00,  1.97it/s]  


In [56]:
df.to_excel('sub_city_latandlon.xlsx',index=False)